# Amazon SageMaker administration and security workshop: Lab 2

This notebook contains hands-on exercises for the workshop **Amazon SageMaker administration and security** – Lab 2.

## Import packages and load variables

In [ ]:
import boto3
import sagemaker
import os
import json
from sagemaker.network import NetworkConfig
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

sagemaker.__version__

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 01-lab-01 notebook         ")
    print("++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "sm-admin-workshop/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
ssm = boto3.client("ssm")
sm_role = sagemaker.get_execution_role()

## Data protection

### Setup parameters
We retrieve the network configuration and encyption key parameters from the SSM store where they were saved by the CloudFormation stacks at provisioning time.

In [ ]:
# Account id and region
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

account_id, region

In [ ]:
security_group_ids = ssm.get_parameter(Name=f"sagemaker-admin-workshop-{region}-{account_id}-sagemaker-sg-ids")["Parameter"]["Value"]
private_subnet_ids = ssm.get_parameter(Name=f"sagemaker-admin-workshop-{region}-{account_id}-private-subnet-ids")["Parameter"]["Value"]
ebs_key_arn = ssm.get_parameter(Name=f"sagemaker-admin-workshop-{region}-{account_id}-kms-ebs-key-arn")["Parameter"]["Value"]

security_group_ids, private_subnet_ids, ebs_key_arn

In [ ]:
# Construct the NetworkConfig with the values for your environment
network_config = NetworkConfig(
        enable_network_isolation=False, 
        security_group_ids=security_group_ids.split(','),
        subnets=private_subnet_ids.split(','),
        encrypt_inter_container_traffic=True)

### Configure SageMaker processing job

In [ ]:
framework_version = "0.23-1"
processing_instance_type = "ml.m5.large"
processing_instance_count = 1

In [ ]:
# Define processing inputs and outputs
processing_inputs = [
        ProcessingInput(
            source=input_s3_url, 
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
]

processing_outputs = [
        ProcessingOutput(
            output_name="train_data", 
            source="/opt/ml/processing/output/train",
            destination=train_s3_url,
        ),
        ProcessingOutput(
            output_name="validation_data", 
            source="/opt/ml/processing/output/validation", 
            destination=validation_s3_url
        ),
        ProcessingOutput(
            output_name="test_data", 
            source="/opt/ml/processing/output/test", 
            destination=test_s3_url
        ),
]

### Enforce encryption of input data
Follow the instructions in the workshop lab 2.
Add the `Deny` inline policy to the user profile execution role using the `ebs_key_arn` you retrieve in the **Setup parameters** section. This inline policy enforces usage of the volume KMS key with the value equal to `ebs_key_arn`.

First, create a processor _without_ volume encryption.

In [ ]:
# Create a processor
sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=sm_role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count, 
    base_job_name='sm-admin-workshop-processing',
    sagemaker_session=sm_session,
    network_config=network_config,
#    volume_kms_key = ebs_key_arn
)

This run will fail because the execution role policy requires usage of the designated volume key, which wasn't provided.

In [ ]:
# Start the processing job
sklearn_processor.run(
        inputs=processing_inputs,
        outputs=processing_outputs,
        code='preprocessing.py',
        wait=True,
)

Second, create a new processor with the intended value of `volume_kms_key` and run the processing job. This time you can call `processor.run()` successfully.

In [ ]:
# Create a processor and specify an EBS volume KMS key
sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=sm_role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count, 
    base_job_name='sm-admin-workshop-processing',
    sagemaker_session=sm_session,
    network_config=network_config,
    volume_kms_key = ebs_key_arn
)

In [ ]:
# This call wil succeed and the processing job will finish
sklearn_processor.run(
        inputs=processing_inputs,
        outputs=processing_outputs,
        code='preprocessing.py',
        wait=True,
)

## Data access control

## Data perimeter

In [ ]:
bucket_name = sagemaker.Session().default_bucket()
bucket_name

In [ ]:
s3_vpc_id = ssm.get_parameter(Name=f"sagemaker-admin-workshop-{region}-{account_id}-s3-vpce-id")["Parameter"]["Value"]

s3_vpc_id

In [ ]:
!aws s3 ls s3://sagemaker-us-east-1-949335012047

In [ ]:
!aws s3 ls s3://sagemaker-eu-central-1-949335012047

## Resource isolation using tags
This section demostrates how to implement resource isolation for user profiles and execution roles using tags. 

<div class="alert alert-info"> ❗ Make sure you attached the pre-provisioned resource isolation inline policy with the name like "sagemaker-admin-workshop-iam-SageMakerTagBasedResourceIsolationPolicy-UUID" to the user profile execution role. For instructions refer to the solutions for the lab 2.
</div>

This example implements isolation for SageMaker processing and training jobs. You can extend the same approach for any taggable resources, for example SageMaker pipelines, model registry model package groups, models, or inference endpoints.

We implement resource isolation based on the value of the tag `team`. Each user profile can tag, create, and describe jobs for it's own team only. 

In [ ]:
team_tag_key = "team"
session = boto3.session.Session()
sm = session.client("sagemaker")

print(f"user profle execution role: {sagemaker.get_execution_role()}")

In [ ]:
# Get the job name, ARN, and tags for the job wth the specified index
def GetJobTags(JobIndex=0):
    r = sm.list_processing_jobs(MaxResults=JobIndex+1)
    if len(r["ProcessingJobSummaries"]) < JobIndex+1:
        print(f"No processing job with the index {JobIndex} found!")
        return {}
    
    processing_job_name = r["ProcessingJobSummaries"][JobIndex]["ProcessingJobName"]
    processing_job_arn =  r["ProcessingJobSummaries"][JobIndex]["ProcessingJobArn"]

    return processing_job_name, processing_job_arn, sm.list_tags(ResourceArn=processing_job_arn)["Tags"]

In [ ]:
# Show the tags of the last processing job
# Make sure you have created at least one processing job in the previous experiments
GetJobTags(0)

In [ ]:
# Get tags for the current user profile
def GetUserProfileTags():
    # Get the tags of the current user profile
    NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"

    # Check what profile you're currently in
    if os.path.exists(NOTEBOOK_METADATA_FILE):
        with open(NOTEBOOK_METADATA_FILE, "rb") as f:
            user_profile_name = json.loads(f.read())['UserProfileName']
            print(f"User profile: {user_profile_name}")

    user_profile_arn = sm.describe_user_profile(DomainId=domain_id, UserProfileName=user_profile_name)["UserProfileArn"]
    return sm.list_tags(ResourceArn=user_profile_arn)["Tags"]

In [ ]:
# Show the tags on the user profile
user_tags = GetUserProfileTags()
user_tags

In [ ]:
# Get the value of the tag "team"
user_team = [v['Value'] for v in user_tags if v['Key'] == team_tag_key][0]
print(f"The team key value for the current user profile: {user_team}")

### Access control for tagging operation

It's important to protect your tags on resources when you implement tag-based access control. For example, you can completely block any access to tags for user execution roles. The resource isolation policy in this example implements a less restrictive approach and enforces the following rules for `AddTags`, `DeleteTags` operations:
- If the resource has the `team` tag with the value _different_ from that of the current user profile, any tagging operation is denied. For example, for the user from the `Team-A`, the policy denies `AddTags` and `DeleteTags` if the resource is tagged with the tag `team` with the value not equal to`Team-A`
- If there is no `team` tag on the resource or the `team` tag value is the same as that of the current user profile, the user can tag the resource
- If the tag `team` is added to the resource, it's value must be the same as that of the current user profile

The following code cells demonstrate these use cases.

In [ ]:
job_name, job_arn, job_tags  = GetJobTags(0)
job_name, job_arn, job_tags

In [ ]:
# Set intended tag key and  value - success
sm.add_tags(ResourceArn=job_arn, Tags=[{'Key': team_tag_key, 'Value': user_team}])

In [ ]:
# Set any other tag key - success
sm.add_tags(ResourceArn=job_arn, Tags=[{'Key': 'another-tag-key', 'Value': 'key-value'}])

In [ ]:
# Set intended tag key and unintended value - AccessDeniedException
sm.add_tags(ResourceArn=job_arn, Tags=[{'Key': team_tag_key, 'Value': 'unitended-value'}])

In [ ]:
# Set intended tag key and uninteded value - AccessDeniedException
sm.add_tags(ResourceArn=job_arn, Tags=[{'Key': team_tag_key, 'Value': 'team-B'}, {'Key': 'another-tag-key', 'Value': 'key-value'}])

In [ ]:
# Print the job tags
GetJobTags(0)

### Access control for `Create*` operations
For tag-based resource isolation works properly you need to tag any new resource at it's creation. For SageMaker processing and training job you must provide correct tags in `CreateProcessingJob` and `CreateTrainingJob` API calls.

Any call of `CreateProcessingJob` must include the tag key `team` with the correct value. The value must be equal to that of the user profile tag key `team`.

In [ ]:
def CreateProcessingJobWithTags(Tags):
    sklearn_processor = SKLearnProcessor(
        framework_version=framework_version,
        role=sm_role,
        instance_type=processing_instance_type,
        instance_count=processing_instance_count, 
        base_job_name='sm-admin-workshop-processing',
        sagemaker_session=sm_session,
        network_config=network_config,
        volume_kms_key = ebs_key_arn,
        tags=Tags
    )
    
    # This call will create a new processing jobs with the specified tags from the Processor class
    sklearn_processor.run(
            inputs=processing_inputs,
            outputs=processing_outputs,
            code='preprocessing.py',
            wait=False,
    )

Now try to create a processing job with different combinations of the tags.

<div class="alert alert-info"> ❗ Make sure you executed Setup parameters and Configure SageMaker processing job in the Data protection section. 
</div>

In [ ]:
# No tags attached
Tags=[]

# This call will fail with AccessDeniedException
CreateProcessingJobWithTags(Tags)

In [ ]:
# Unintended tag key
Tags=[{'Key': 'unintended-key', 'Value': user_team}]

# This call will fail with AccessDeniedException
CreateProcessingJobWithTags(Tags)

In [ ]:
# Unintended tag value
Tags=[{'Key': team_tag_key, 'Value': 'unintended-value'}]

# This call will fail with AccessDeniedException
CreateProcessingJobWithTags(Tags)

In [ ]:
# The only correct tag key and value
Tags=user_tags
Tags

In [ ]:
# This call will succed and create a processing job
CreateProcessingJobWithTags(Tags)

### Access control for `Describe, Delete, Update, Stop` operations
The resource isolation policy denies all operations on the protected resource if the intended tag missing or set to a value that doesn't match that of the current user profile.
These operations are only allowed on the jobs tagged with the tag `team` with the value equal to that of the user profile tag key `team`.

In [ ]:
session = boto3.session.Session()
sm = session.client("sagemaker")

In [ ]:
# Print the last job tags
job_name, job_arn, job_tags = GetJobTags(0)
job_name, job_arn, job_tags

In [ ]:
# Print the tags on the user profile
GetUserProfileTags()

In [ ]:
# Succeds only if the tags on the user profile and the job match
sm.describe_processing_job(ProcessingJobName=job_name)

In [ ]:
# Access to StopProcessingJob API is only allowed if tags match
sm.stop_processing_job(ProcessingJobName=job_name)

### Access resource from a different user profile
Now open a new browser and sign in with a different user profile with a different  execution role. The execution role for that user profile has a different value of the `team` tag. Run this section in the new Studio. Make sure you have a different user profile name shown in the right corner of the Studio.

The calls to `DescribeProcessingJob` and `StopProcessingJob` will fail because the tag value on the user profile doesn't match the tag value on the processing job.

Congratulations, you've just implemented the resource isolation using tags and IAM permission policies. Feel free to experiment further with other resources and more sophisticated isolation rules.

### Clean up
Remove the resource isolation policy from the user profile execution roles.
You can use the generated link to the AWS IAM console in the next cell.

In [ ]:
from IPython.core.display import display, HTML

# Execute this cell to show the execution role IAM console link
display(
    HTML(
        '<b>Remove the resource isolation policy from the execution role in <a target="top" href="https://us-east-1.console.aws.amazon.com/iamv2/home#/roles/details/{}?section=permissions">AWS IAM console.</a></b>'.format(
            sagemaker.get_execution_role().split("/")[-1])
    )
)

## End of the lab 2
Follow the instructions in the lab 3 of the workshop and the [`03-lab-03.ipynb`](03-lab-03.ipynb) notebook.

---

## Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>